In [1]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")

In [2]:
overlay.load_model("./dpu_cf_mobilenetv2.elf")

In [3]:
from ctypes import *
import cv2
import numpy as np
from dnndk import n2cube
import os
import threading
import time
from pynq_dpu import dputils

KERNEL_CONV = "cf_mobilenetv2"
KERNEL_CONV_INPUT = "263"
KERNEL_FC_OUTPUT = "417"
num_iterations = 500

In [4]:
from IPython.display import display
from PIL import Image

img = cv2.imread("/home/xilinx/holdout_images/ILSVRC2012_val_00000001.JPEG")

In [5]:
n2cube.dpuOpen()
kernel = n2cube.dpuLoadKernel(KERNEL_CONV)

In [6]:
task = n2cube.dpuCreateTask(kernel, 0)
mean = [104,117,123]
dputils.dpuSetInputImageWithScale(task, KERNEL_CONV_INPUT, img, mean, 0.00390625) 
n2cube.dpuGetInputTensor(task, KERNEL_CONV_INPUT)

In [7]:
n2cube.dpuRunTask(task)
size = n2cube.dpuGetOutputTensorSize(task, KERNEL_FC_OUTPUT)
channel = n2cube.dpuGetOutputTensorChannel(task, KERNEL_FC_OUTPUT)
conf = n2cube.dpuGetOutputTensorAddress(task, KERNEL_FC_OUTPUT)
outputScale = n2cube.dpuGetOutputTensorScale(task, KERNEL_FC_OUTPUT)
resultFP = n2cube.dpuGetOutputTensorInHWCFP32(task, KERNEL_FC_OUTPUT, size)
resultINT = n2cube.dpuGetOutputTensorInHWCInt8(task, KERNEL_FC_OUTPUT, size)
softmax = n2cube.dpuRunSoftmax(conf, channel, size//channel, outputScale)

In [8]:
outputScale

0.25

In [9]:
resultINT

array([  3,  -4,   9,   3,  15,  18,  21,   3,   3,   3,   3,  -8, -17,
        -3,  -5,  -5, -13,   0,  -7,   2,  -1,  -3,  -7,  -4,   6,  12,
        21,  -9,  11,   4,  18,  -4,  13,  43,  53,  23,  30,  16,  21,
        28,  -1,  26,  21,  16,  32,  31,  10,  18,  29,  48,  49,   5,
        20,  28,  54,  14,  37,  34,  45,  17,  40,  39,  58,  33,  13,
        45,  41,  43,  41,  18,  -2,  14,  -9,  -4,  -4, -13,   3,   2,
        -6,  13,  -6,  11,  14,   7,  16,  22,  13, -10, -10,  -9, -17,
        -3,  -7,   1,  -5, -10, -11,   3,   3,  -5,   0,   4,   7,   7,
         2,  -9,   0,  13,   9,   3,  12,  10,  23,  18,  30,   8,  21,
         8,   4,  15,  18,   3,  -1,   5,   9,  18,   6, -13, -17, -10,
       -11,  -1,  -1,   9, -13,  -1, -12,   2,  13,  10,  13,  10,  14,
       -13,  -5,   3,  -8,  15,   9,   2,  19,   2,  -4,   7,   8,   9,
        -9,   0,  -6,   0,   4,  -2,  -3,  -4,  10, -12,  -8, -13,  -7,
         1,   6,  -5,   2, -11,  -1,  -7,   2,   7,  -8,  -2,  -

In [9]:
with open("./words.txt", "r") as f:
    lines = f.readlines()

In [10]:
print('Class label: {}'.format(lines[np.argmax(softmax)]))

Class label: rock python, rock snake, Python sebae



In [10]:
!dexplorer -s

[DPU mode]
normal

[DPU timeout limitation (in seconds)]
10

[DPU Debug Info]
Core 0 schedule : 1
Core 0 interrupt: 1

[DPU Resource]
DPU Core  	: 0
State     	: Idle
PID       	: 4687
TaskID    	: 4687
Start     	: 1610162931856138000
End       	: 1610162931866336000

[DPU Core 0 Register]
CTL       : 0x00000004
GIE       : 0x00000000
IRQ       : 0x00000000
HP        : 0x07070f0f
CODE      : 0x0000000000078280
BASE0     : 0x0000000078300000
BASE1     : 0x0000000078700000
BASE2     : 0x0000000000000000
BASE3     : 0x0000000000000000
BASE4     : 0x0000000000000000
BASE5     : 0x0000000000000000
BASE6     : 0x0000000000000000
BASE7     : 0x0000000000000000
CYCLE_H   : 0x00000000
CYCLE_L   : 0x00000000
REGVER    : 0x32016b02
TIMESTAMP : 0x1431a0d2
GITID     : 0x07b7683a
GITTIME   : 0x77675797
VERSION   : 0x00000141
TIMER     : 0x00000000
ARCH      : 0x31280a0a
RAM       : 0x00001333
LOAD      : 0x00000002
CONV      : 0x00000111
SAVE      : 0x00000002
POOL      : 0x00000001
ELEW      : 0x0